In [19]:
from torch_geometric.data import HeteroData
import pickle
import numpy as np
import torch

In [20]:
# data = HeteroData()

# data['paper'].x = ... # [num_papers, num_features_paper]
# data['author'].x = ... # [num_authors, num_features_author]
# data['institution'].x = ... # [num_institutions, num_features_institution]
# data['field_of_study'].x = ... # [num_field, num_features_field]

# data['paper', 'cites', 'paper'].edge_index = ... # [2, num_edges_cites]
# data['author', 'writes', 'paper'].edge_index = ... # [2, num_edges_writes]
# data['author', 'affiliated_with', 'institution'].edge_index = ... # [2, num_edges_affiliated]
# data['paper', 'has_topic', 'field_of_study'].edge_index = ... # [2, num_edges_topic]


In [21]:
num_diagnosis = 2042
num_process = 591

edge_attr = np.load("data/edge_attr.npy")
edge_index = np.load("data/edge_index.npy")
node_label = np.load("data/node_label.npy")
with open("data/node2feat.pickle", "rb") as f:
    node2feat = pickle.load(f)

diagnosis_edge_index = [[], []]
process_edge_index = [[], []]
for eidx in range(edge_index.shape[1]):
    atr = edge_attr[eidx]
    if atr == "diagnosis":
        diagnosis_edge_index[0].append(edge_index[0][eidx])
        diagnosis_edge_index[1].append(edge_index[1][eidx])
    elif atr == "process":
        process_edge_index[0].append(edge_index[0][eidx])
        process_edge_index[1].append(edge_index[1][eidx])
self_edge_index = [[],[]]
for vid in node2feat.keys():
    self_edge_index[0].append(vid)
    self_edge_index[1].append(vid)
        
train_mask = np.load("data/train_mask.npy")
val_mask = np.load("data/val_mask.npy")
test_mask = np.load("data/test_mask.npy")

In [26]:
from torch_geometric.data import HeteroData
import pickle

data = HeteroData()

data['icd9'].x = torch.FloatTensor([[v] for v in range(num_process)])
data['ndc'].x = torch.FloatTensor([[v] for v in range(num_diagnosis)])
data['patient'].x = torch.FloatTensor([node2feat[v] for v in sorted(list(node2feat.keys()))])

data['patient', 'process', 'icd9'].edge_index = torch.LongTensor(process_edge_index)
data['icd9', 'process', 'patient'].edge_index = torch.LongTensor([process_edge_index[1], process_edge_index[0]])
data['patient', 'diagnosis', 'ndc'].edge_index = torch.LongTensor(diagnosis_edge_index)
data['patient', 'diagnosis', 'ndc'].edge_index = torch.LongTensor([diagnosis_edge_index[0], diagnosis_edge_index[1]])
data['patient', 'self', 'patient'].edge_index = torch.LongTensor(self_edge_index)

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

In [27]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import HGTConv, Linear

class HGT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_heads, num_layers):
        super().__init__()

        self.lin_dict = torch.nn.ModuleDict()
        for node_type in data.node_types:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, data.metadata(),
                           num_heads, group='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for node_type, x in x_dict.items():
            x_dict[node_type] = self.lin_dict[node_type](x).relu_()

        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)

        return self.lin(x_dict['patient'])

In [28]:
model = HGT(64, 1, 4, 2)

In [29]:
model(data.x_dict, data.edge_index_dict)

IndexError: Encountered an index error. Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 5183] (got interval [1, 5694])